**Assignment 6**

**Image and Video Analytics**

**Vishali Sharma**

**21MIA1066**

Task 6: Spotting and Counting a Branded Car in a Video

Objective:
Identify and count the number of branded cars (e.g., a specific logo or color) in a video sequence using feature-based matching.

**Task Description:**

**Load Video:**

**Load the provided video showing vehicles.**

**Car Detection:**

**Use background subtraction or motion detection to detect moving cars in the video.**

**Feature Matching:**

**Use color-based detection or template matching to identify the specific branded car (e.g., a car with a specific color or logo).**

**Track the occurrence of this branded car across the video frames.**

**Counting:**

**Count the number of times the branded car appears in the video.**

**Result:**

**Output the total count and display the frames where the branded car is detected.**

In [ ]:
import cv2
import numpy as np

# Path to the uploaded video
video_path ="task_6.mp4"

# Load the video
cap = cv2.VideoCapture(video_path)

# Check if video loaded successfully
if not cap.isOpened():
    raise Exception("Error: Could not open video file.")

# Background subtractor for motion detection
fgbg = cv2.createBackgroundSubtractorMOG2()

# Define refined color ranges for black, white, brown, and red cars in HSV color space
color_ranges = {
    "black": ((0, 0, 0), (180, 500, 40)),  # Restricting upper limit for black
    "white": ((0, 0, 200), (180, 30, 255)),  # Narrowing white's brightness threshold
    "brown": ((10, 100, 50), (20, 255, 200)),  # More precise for brown range
    "red1": ((0, 70, 50), (10, 255, 255)),  # First range for red
    "red2": ((160, 70, 50), (180, 255, 255)),  # Second range for red
}

# Initialize car count
car_count = 0

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Apply background subtraction to detect moving objects
    fgmask = fgbg.apply(frame)
    _, fgmask = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
    
    # Find contours of the detected areas
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter and process each contour
    for cnt in contours:
        if cv2.contourArea(cnt) < 500:  # Minimum area to filter noise
            continue
        
        # Get bounding box for each detected car
        x, y, w, h = cv2.boundingRect(cnt)
        car_roi = frame[y:y+h, x:x+w]  # Region of interest (ROI) for the car
        
        # Convert ROI to HSV color space
        hsv_roi = cv2.cvtColor(car_roi, cv2.COLOR_BGR2HSV)
        
        # Check for branded car color match and set label
        color_label = None
        for color, (lower, upper) in color_ranges.items():
            if color == "red2":
                # Second range for red color (as red wraps in HSV)
                mask = cv2.inRange(hsv_roi, color_ranges["red1"][0], color_ranges["red1"][1]) | \
                       cv2.inRange(hsv_roi, color_ranges["red2"][0], color_ranges["red2"][1])
            else:
                mask = cv2.inRange(hsv_roi, np.array(lower), np.array(upper))
            
            # Check if a sufficient area within the ROI matches the color
            if cv2.countNonZero(mask) > (0.2 * w * h):  # At least 20% of the area matches
                color_label = color
                car_count += 1
                break

        # Draw bounding box and color label if a branded car was detected
        if color_label:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, color_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display the frame with detections
    cv2.imshow('Branded Car Detection', frame)
    
    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

# Print the total count of detected branded cars
print(f"Total car detections: {car_count}")